In [3]:
# word2vector 로 유사어 찾기

import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [13]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [18]:
df = pd.read_csv('C:/Users/USER/Desktop/project/ML_study/archive/CommentsApril2017.csv')
df.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_9444\2176771596.py:1: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/USER/Desktop/project/ML_study/archive/CommentsApril2017.csv')


,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


In [37]:
raw_text = df.commentBody
print(raw_text)
normalized_text = []
for string in raw_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

0         This project makes me happy to be a 30+ year T...
1         Stunning photos and reportage. Infuriating tha...
2         Brilliant work from conception to execution. I...
3         NYT reporters should provide a contributor's l...
4            Could only have been done in print. Stunning. 
                                ...                        
243827    Sorry, but pudding has nothing to do with it; ...
243828    While it would be quite punny to spell it "des...
243829         See above comments. "deserts" is the proper.
243830    John Rubinstein had two brief scenes with Joan...
243831    Ye gods and little fishes, Stu, you're right ....
Name: commentBody, Length: 243832, dtype: object


In [38]:
print(normalized_text[:5])

['this project makes me happy to be a 30 year times subscriber continue to innovate across all platforms please ', 'stunning photos and reportage infuriating that the trump admistration s draconian reinstatement of the global gag order will prevent men and women from receiving appropriate family planning advice so obviously desperately needed ', 'brilliant work from conception to execution i ve never seen anything like it as a paper of record it is important to leave a record of these and other peoples for whom no one else speaks please keep humanizing what are otherwise bland statistics it matters time and again their stories sounded like mine could have been mine we re about to see hordes of people far away experiencing utter devastation empathy goes away unless there are names and stories and pictures links to helping at organizations resources ', 'nyt reporters should provide a contributor s link to a crowdfunding project or aid organization focused on the specific subject of news 

In [39]:
result = [word_tokenize(sentence) for sentence in normalized_text]

In [40]:
print(result[0:5])

[['this', 'project', 'makes', 'me', 'happy', 'to', 'be', 'a', '30', 'year', 'times', 'subscriber', 'continue', 'to', 'innovate', 'across', 'all', 'platforms', 'please'], ['stunning', 'photos', 'and', 'reportage', 'infuriating', 'that', 'the', 'trump', 'admistration', 's', 'draconian', 'reinstatement', 'of', 'the', 'global', 'gag', 'order', 'will', 'prevent', 'men', 'and', 'women', 'from', 'receiving', 'appropriate', 'family', 'planning', 'advice', 'so', 'obviously', 'desperately', 'needed'], ['brilliant', 'work', 'from', 'conception', 'to', 'execution', 'i', 've', 'never', 'seen', 'anything', 'like', 'it', 'as', 'a', 'paper', 'of', 'record', 'it', 'is', 'important', 'to', 'leave', 'a', 'record', 'of', 'these', 'and', 'other', 'peoples', 'for', 'whom', 'no', 'one', 'else', 'speaks', 'please', 'keep', 'humanizing', 'what', 'are', 'otherwise', 'bland', 'statistics', 'it', 'matters', 'time', 'and', 'again', 'their', 'stories', 'sounded', 'like', 'mine', 'could', 'have', 'been', 'mine', 'we

In [41]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 243832


In [42]:
# 벡터 차원 100, 주변단어 범위 5, 최소 빈도 5, 프로세스 수 4, CBOW 방식으로 학습
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [43]:
#'man' 유사 단어 

model_result = model.wv.most_similar("man")
print(model_result)

[('person', 0.8251510858535767), ('guy', 0.816261351108551), ('woman', 0.7441701292991638), ('artist', 0.6908255815505981), ('gentleman', 0.685021698474884), ('charlatan', 0.6590436100959778), ('buffoon', 0.6422693729400635), ('boy', 0.6412042379379272), ('psychopath', 0.6332657337188721), ('creature', 0.6260451674461365)]


In [44]:
#'rain' 유사 단어 

model_result = model.wv.most_similar("rain")
print(model_result)

[('snow', 0.8498326539993286), ('forest', 0.8097003698348999), ('trees', 0.7840538024902344), ('sun', 0.7613822817802429), ('winter', 0.7404351234436035), ('waves', 0.7399057745933533), ('dust', 0.7229664325714111), ('fires', 0.7211699485778809), ('desert', 0.7167099714279175), ('melt', 0.7158868312835693)]


In [45]:
# 해당 모델 저장

model.wv.save_word2vec_format('eng_w2v_01') # 모델 저장